In [14]:
import pandas as pd
import random
import wordninja

In [15]:
pos_locations = ['positive data/spring-boot_labeled_sentences.csv','positive data/facebook_labeled_sentences.csv',
                 'positive data/tensorflow_labeled_sentences.csv','positive data/vscode_labeled_sentences.csv',
                 'positive data/flutter_labeled_sentences.csv']
q_locations = ['data/spring-boot_PRs.txt','data/facebook_PRs.txt',
               'data/tensorflow_PRs.txt','data/vscode_PRs.txt',
               'data/flutter_PRs.txt']

In [16]:
def get_dataset(loc1,loc2):
    df = pd.read_csv(loc1)
    df2 = pd.read_table(loc2, sep = ',')
    issue_id = df.values[:,1].tolist()
    lbl_lst = df.values[:,3].tolist()
    all_issue = df2.values[:,1].tolist()
    question_lst = df2.values[:,11].tolist()

    newdic = dict(zip(issue_id,lbl_lst))
    pos_issue = list(set([i for i,j in newdic.items() if j == 0]))
    all_neg_issue = list(set([x for x in all_issue if x not in issue_id]))
    neg_issue = random.sample(all_neg_issue,len(pos_issue))
    pos_question = [q for q,i in zip(question_lst,all_issue) if i in pos_issue]
    neg_question = [q for q,i in zip(question_lst,all_issue) if i in neg_issue]
    question_lst= pos_question + neg_question
    lable_lst = [1 for i in range(len(pos_question))] + [0 for i in range(len(neg_question))]
    
    return question_lst,lable_lst


In [17]:
question_list = []
lable_list = []
for loc1,loc2 in zip(pos_locations,q_locations):
    q1,l1 = get_dataset(loc1,loc2)
    question_list+=q1
    lable_list+=l1
    
print(len(question_list))
print(len(lable_list))


29853
29853


In [18]:
labeled_question = pd.DataFrame({'question': question_list,'label':lable_list})

print(labeled_question.shape)
labeled_question.dropna(axis=0,how='any',inplace=True)
labeled_question['question'] = labeled_question['question'].str.lower()
print(labeled_question.shape)
labeled_question.head()


(29853, 2)
(28912, 2)


,question,label
0,<!-- thanks for raising a spring boot issue. p...,1
1,my code is like this ```java @configuration @...,1
3,can i use a higher version of the `spring-boot...,1
4,i only found the documentation about authentic...,1
5,"hi, i deployed a project made with https://sta...",1


Data Cleaning

In [19]:
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re, string, unicodedata

In [20]:
def html_remover(data):
  beauti = BeautifulSoup(data,'html.parser')
  return beauti.get_text()

# to remove URL
def url_remover(data):
  return re.sub(r'https\S','',data)

def web_associated(data):
  text = html_remover(data)
  text = url_remover(text)
  return text

In [21]:
def remove_round_brackets(data):
  return re.sub('\(.*?\)','',data)

def remove_punc(data):
  trans = str.maketrans('','', string.punctuation)
  return data.translate(trans)

def white_space(data):
  return ' '.join(data.split())

def complete_noise(data):
  new_data = remove_round_brackets(data)
  new_data = remove_punc(new_data)
  new_data = white_space(new_data)
  return new_data

In [22]:
def Data_cleaning(x):
    return complete_noise(web_associated(x))

for index,row in labeled_question.iterrows():
    text = Data_cleaning(row['question'])
    labeled_question.loc[index,'question'] = text

cleaned_labeled_question = labeled_question.loc[labeled_question['question'].str.len()>10]
cleaned_labeled_question.head()


c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use

,question,label
1,my code is like this java configuration enable...,1
3,can i use a higher version of the springbootma...,1
4,i only found the documentation about authentic...,1
5,hi i deployed a project made with startspringi...,1
6,we have existing couple of spring applications...,1


In [ ]:
def split_word(df):
    df = df.apply(lambda x: ' '.join(wordninja.split(x)))
    
cleaned_labeled_question = split_word(cleaned_labeled_question['question'])

In [ ]:
cleaned_labeled_question.loc[1,'question']

In [ ]:
cleaned_labeled_question['label'].value_counts()

Output Data

In [ ]:
# cleaned_labeled_question.to_csv('labeled_question_for_all.csv', index=False, encoding='utf-8')

In [ ]:
# small_sample = cleaned_labeled_question.sample(3000)
# small_sample.to_csv('labeled_question_small_sample.csv', index=False, encoding='utf-8')